#  Прогноз добычи нефти

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

 Дано три датасета с данными геологоразведки трёх регионов: 
1. id — уникальный идентификатор скважины;
2. f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
3. product — объём запасов в скважине (тыс. баррелей).

   Путь к данным
1. df0: /datasets/geo_data_0.csv
2. df1: /datasets/geo_data_1.csv
3. df2: /datasets/geo_data_2.csv

   Условия задачи:
   1. Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
   2. При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
   3. Бюджет на разработку скважин в регионе — 10 млрд рублей.
   4. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
   5. После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка данных 



In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

In [2]:
# Загружаем датасеты
df0 = pd.read_csv('/datasets/geo_data_0.csv', sep = ',')
df1 = pd.read_csv('/datasets/geo_data_1.csv', sep = ',')
df2 = pd.read_csv('/datasets/geo_data_2.csv', sep = ',')

# Для каждого датасета проверяем дубликаты и смотрим общую информацию
print('df0:')
print('Дубликаты df0:', df0.duplicated().sum())
print(df0.info())
print('df1:')
print('Дубликаты df1:', df1.duplicated().sum())
print(df1.info())
print('df2:')
print('Дубликаты df2:', df2.duplicated().sum())
print(df2.info())

df0:
Дубликаты df0: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
df1:
Дубликаты df1: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
df2:
Дубликаты df2: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000

In [3]:
pd.options.display.float_format = '{:,.2f}'.format
# Для каждого датасета исключаем мультиколлинеарность
print('Region 1')
df0.corr()

Region 1


,f0,f1,f2,product
f0,1.00,-0.44,-0.00,0.14
f1,-0.44,1.00,0.00,-0.19
f2,-0.00,0.00,1.00,0.48
product,0.14,-0.19,0.48,1.00


In [4]:
print('Region 2')
df1.corr()

Region 2


,f0,f1,f2,product
f0,1.00,0.18,-0.00,-0.03
f1,0.18,1.00,-0.00,-0.01
f2,-0.00,-0.00,1.00,1.00
product,-0.03,-0.01,1.00,1.00


In [5]:
print('Region 3')
df2.corr()

Region 3


,f0,f1,f2,product
f0,1.00,0.00,-0.00,-0.00
f1,0.00,1.00,0.00,-0.00
f2,-0.00,0.00,1.00,0.45
product,-0.00,-0.00,0.45,1.00


Выводы:
1. Явных дубликатов нет.
2. Пропусков нет, типы данных в норме.
3. Корреляции между признаками нет - мультиколлинеарности нет. Корелляция между столбцами 'f2' и 'product' - корелляция между признаком и цедевым значением. Это не навредит, а наоборот поможет модели обучаться.

## Подготовка данных. Обучение и проверка модели для каждого региона.

In [6]:
# Удалим бесполезный столбец
df0.drop(['id'], axis=1, inplace=True)
df1.drop(['id'], axis=1, inplace=True)
df2.drop(['id'], axis=1, inplace=True)

In [7]:
# Создадим функцию для датасетов на выборки и обучения модели.
def process(data):
    # Разделение датасетов на выборки
    features = data.drop(['product'], axis=1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
    # Масштабируем признаки
    pd.options.mode.chained_assignment = None
    scaler = StandardScaler()
    scaler.fit(features_train[['f0', 'f1', 'f2']])
    features_train[['f0', 'f1', 'f2']] = scaler.transform(features_train[['f0', 'f1', 'f2']])
    features_valid[['f0', 'f1', 'f2']] = scaler.transform(features_valid[['f0', 'f1', 'f2']])
    
    # Обучим модель и получим предсказания среднего запаса сырья в скважине
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    predicted_valid = pd.Series(predicted_valid, index = target_valid.index)
    rmse = mean_squared_error(target_valid, predicted_valid, squared=False)
    
    print('Средний запас предсказанного сырья: {0:.2f}'.format(predicted_valid.mean()))
    print("RMSE = {0:.2f}".format(rmse))
    
    return predicted_valid, target_valid, rmse

# Применим функцию, сохраним результаты и выведем получившиеся данные
print('Region 1:')
predicted0_valid, target0_valid, rmse0 = process(df0)
print()
print('Region 2:')
predicted1_valid, target1_valid, rmse1 = process(df1)
print()
print('Region 3:')
predicted2_valid, target2_valid, rmse2 = process(df2)


Region 1:
Средний запас предсказанного сырья: 92.59
RMSE = 37.58

Region 2:
Средний запас предсказанного сырья: 68.73
RMSE = 0.89

Region 3:
Средний запас предсказанного сырья: 94.97
RMSE = 40.03


    Вывод:
    
    По результатам моделирования видим,что средний запас сырья в скважине больше в 3ем регионе и примерно такой же в 1ом, однако модель для второго региона более точная rmse = 0.89. Скорее всего в связи с тем, что корреляция между F2 и  product во втором наборе данных 100%, модель для второго региона точнее.

## Подготовка к расчету прибыли.

In [8]:
# Сохраним значения для расчетов в отдельные переменные
budget = 10**10
income = 450000
points = 200
# Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины
print(
    'Средний объем сырья новой скважины для безубыточной разработки: {0:.2f}'.format(budget/(income*points)),
    "тыс. баррелей."
)


Средний объем сырья новой скважины для безубыточной разработки: 111.11 тыс. баррелей.


По предсказанным средним запасам сырья в скважине в регионах ни один из них не подходит при сравнении со значением достаточного объёма сырья для безубыточной разработки новой скважины.

## Расчет прибыли по выбранным скважинам и предсказаниям модели. 

In [9]:
# Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели
def profit_reg(predicts, target):
    # Отсортируем скважины с максимальными значениями предсказаний
    probs_sorted = predicts.sort_values(ascending=False)
    selected = target[probs_sorted.index][:points]
    # Рассчитаем прибыль для полученного объёма сырья
    profit = selected.sum()*income - budget
    return profit

print('Прибыль для region 1, руб.:', profit_reg(predicted0_valid, target0_valid))
print('Прибыль для region 2, руб.:', profit_reg(predicted1_valid, target1_valid))
print('Прибыль для region 3, руб.:', profit_reg(predicted2_valid, target2_valid))



Прибыль для region 1, руб.: 3320826043.1398506
Прибыль для region 2, руб.: 2415086696.681511
Прибыль для region 3, руб.: 2710349963.5998325


## Подсчет прибылей и рисков убытков для каждого региона

In [10]:
# Создадим функцию для подсчета прибылей/рисков по регионам
def boostrap(predicts, target):
    # Применим технику Bootstrap с 1000 выборок
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = predicts[target_subsample.index] 
        bootstrap_dif = profit_reg(probs_subsample, target_subsample)
        values.append(bootstrap_dif)

    values = pd.Series(values)
    # Найдем среднюю прибыль
    mean = values.mean() 
    print("Средняя прибыль, руб.:", mean)
    
    # Рассчитаем 95%-й доверительный интервал
    confidence_interval = values.quantile([0.025, 0.975])
    print("95%-ый доверительный интервал:", confidence_interval)
   
    # Определим риск убытков
    risk = (values < 0).mean()*100
    print('Риск убытков,%:', risk)
    
    if risk < 2.5:
        print('Регион подходит для разработки')
    else:
        print('Регион не подходит для разработки')

    return 

# Выведем результаты функции
print('Region 1:')
boostrap(predicted0_valid, target0_valid)
print()
print('Region 2:')
boostrap(predicted1_valid, target1_valid)
print()
print('Region 3:')
boostrap(predicted2_valid, target2_valid)

Region 1:
Средняя прибыль, руб.: 425938526.91059244
95%-ый доверительный интервал: 0.03   -102,090,094.84
0.97    947,976,353.36
dtype: float64
Риск убытков,%: 6.0
Регион не подходит для разработки

Region 2:
Средняя прибыль, руб.: 515222773.4432899
95%-ый доверительный интервал: 0.03    68,873,225.37
0.97   931,547,591.26
dtype: float64
Риск убытков,%: 1.0
Регион подходит для разработки

Region 3:
Средняя прибыль, руб.: 435008362.7827556
95%-ый доверительный интервал: 0.03   -128,880,547.33
0.97    969,706,954.18
dtype: float64
Риск убытков,%: 6.4
Регион не подходит для разработки


Вывод: Для разработки предлагается выбрать 2ой регион, так как средняя прибыль наибольшая, а риск убытков - минимальный.